In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1J8brwnHgopc8k_il49FEAB_3BdQvdCAU'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('lib.zip')

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./lib/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "lib.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("lib")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [0]:
id = '1zd_JQpXU6AEi_x1xeTp1ryM4G2k59LVE'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hymenoptera_data.zip')

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "hymenoptera_data.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("data")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [9]:
cd /content

/content


*モデルの適用*

In [10]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.549s
     |████████████████████████████████| 686kB 13.4MB/s 
     |████████████████████████████████| 204kB 52.9MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=e20e862e4cb6c0c6cb0403281793e91c0ad71094a2ffaaa6a8932a00fa167a91
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=be3231f2f737363ba0c9a20335629de07f30ce9cc51022ee8395af94b24cc5c7
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 129 ms, sys: 31 ms, total: 160 ms
Wall time: 22.4 s


In [11]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 14.7 ms, sys: 7.89 ms, total: 22.6 ms
Wall time: 7.17 s


In [0]:
from lib.lib import network_finetuning as network
from lib.lib import architecture
from lib.lib import dataloader as dl
from lib.lib import optimizer
from lib.lib import trainer

import torchvision

In [13]:
%%time
net = network.ResNet50(class_size=2)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
%%time
batch_size = 128
train_dataset = torchvision.datasets.ImageFolder(root='./data/hymenoptera_data/train', transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.ImageFolder(root='./data/hymenoptera_data/val', transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

CPU times: user 1.98 ms, sys: 1.73 ms, total: 3.71 ms
Wall time: 9.24 ms


In [15]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 0 ns, sys: 34.9 ms, total: 34.9 ms
Wall time: 68.1 ms


In [16]:
%%time
model.train(data_loader, epoch_count=50, is_inception=False)

cuda


Setting up a new session...


----------
Epoch 1/50 2020-01-23 12:06:03.481745
val Loss: 2.9257 Acc: 0.5425
----------
Epoch 2/50 2020-01-23 12:06:05.254494
train Loss: 3.0595 Acc: 0.2500
val Loss: 1.1681 Acc: 0.4575
----------
Epoch 3/50 2020-01-23 12:06:10.687744
train Loss: 1.4958 Acc: 0.5205
val Loss: 0.7312 Acc: 0.4575
----------
Epoch 4/50 2020-01-23 12:06:16.022204
train Loss: 0.8318 Acc: 0.5123
val Loss: 0.8224 Acc: 0.4575
----------
Epoch 5/50 2020-01-23 12:06:21.345530
train Loss: 0.9690 Acc: 0.4795
val Loss: 0.6940 Acc: 0.5359
----------
Epoch 6/50 2020-01-23 12:06:26.829896
train Loss: 0.7932 Acc: 0.2951
val Loss: 0.7253 Acc: 0.5425
----------
Epoch 7/50 2020-01-23 12:06:32.226212
train Loss: 0.6331 Acc: 0.6475
val Loss: 0.7786 Acc: 0.5425
----------
Epoch 8/50 2020-01-23 12:06:37.554482
train Loss: 0.5372 Acc: 0.9098
val Loss: 0.8032 Acc: 0.5425
----------
Epoch 9/50 2020-01-23 12:06:42.878910
train Loss: 0.4429 Acc: 0.9385
val Loss: 0.8309 Acc: 0.5425
----------
Epoch 10/50 2020-01-23 12:06:48.180496


In [17]:
%%time
model.predict(data_loader['val'])

検証画像に対しての正解率： 49 %
CPU times: user 278 ms, sys: 267 ms, total: 545 ms
Wall time: 1.71 s
